In [1]:
import random
import matplotlib as mpl
import numpy as np

import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F

In [19]:
class CNN(nn.Module):
    def __init__(self, input_dim=1, output_dim=10):
        super().__init__()
        
        self.convs = nn.Sequential(
            nn.Conv2d(input_dim, 64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=3, stride=1),
            
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=3, stride=1),
            
            nn.Conv2d(64, 32, kernel_size=3, stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            
            nn.AvgPool2d(kernel_size=3, stride=1),
        )
        
        self.linears = nn.Sequential(
            nn.Linear(8192, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2),
            nn.Linear(128, output_dim)
        )
        
    def forward(self, x):
        x = self.convs(x)
        x = torch.flatten(x, 1)
        x = self.linears(x)
        output = F.log_softmax(x, dim=1)
        return output

In [20]:
from torchvision import datasets, transforms

transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,batch_size=32)
test_loader = torch.utils.data.DataLoader(dataset2,batch_size=32)

batch = next(iter(train_loader))
print(batch[0].size())

torch.Size([32, 1, 28, 28])


In [23]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [25]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

device = torch.device("cuda")
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

epochs = 3
for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.329754

Test set: Average loss: 0.0675, Accuracy: 9815/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.079693

Test set: Average loss: 0.0708, Accuracy: 9806/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.011610

Test set: Average loss: 0.0427, Accuracy: 9883/10000 (99%)

